### Links
- PyTorch MNIST example - https://github.com/pytorch/examples/blob/master/mnist/main.py
- pytorch cifar10 code - https://github.com/kuangliu/pytorch-cifar/blob/master/main.py
- LeNet for CIFAR-10 - https://github.com/kuangliu/pytorch-cifar/blob/master/models/lenet.py

My Google Doc - https://docs.google.com/document/d/1nPvFULraomLmBp4IuNVe51YXV1DQsSzgQWtuaZhbkjo/edit
Trello - https://docs.google.com/document/d/1nPvFULraomLmBp4IuNVe51YXV1DQsSzgQWtuaZhbkjo/edit
Using webcam - https://docs.google.com/document/d/17BVx5yioNaV11mip4U9gRAO7wLIJf-aOpIITJPjqeks/edit#
Installing pytorch - https://docs.google.com/document/d/1NfLbuhoOxonzS9FEeooyqpB90-62Gat-uZ2DOTaWRdg/edit#


In [1]:
import argparse
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
print(torch.__version__)
print(torch.__file__)

1.0.1.post2
/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/__init__.py


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, 1, padding=2)  # in_channels, out_channels, kernel_size, stride
        self.conv2 = nn.Conv2d(6, 16, 5, 1)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # in_features, out_features
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.avg_pool2d(x, 2, 2)  # input, kernel_size, stride
        x = F.relu(self.conv2(x))
        x = F.avg_pool2d(x, 2, 2)
        # reshape for fully connected layers...
#         print(x.size())
        x = x.view(-1, 16*5*5)  # what's the shape after this? 1 x 400?
#         print(x.size())
        x = F.relu(self.fc1(x))
#         print(x.size())
        x = F.relu(self.fc2(x))
#         print(x.size())
        x = self.fc3(x)
#         print(x.size())
        x = F.log_softmax(x, dim=1)
#         print(x.size())
        return x


In [4]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')


_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='how many batches to wait before logging training status', metavar='N')

In [ ]:
args = parser.parse_args(args=["--epochs=100"])
args

Namespace(batch_size=64, epochs=100, log_interval=10, lr=0.01, momentum=0.5, no_cuda=False, seed=1, test_batch_size=1000)

In [ ]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda

False

In [ ]:
torch.manual_seed(args.seed)

In [ ]:
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [ ]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
kwargs

{}

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [ ]:
model = Net().to(device)
model

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # to do: needed?
        optimizer.zero_grad()  # what does this do? zero-out any previous gradient info?
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            digits = int(math.log10(len(train_loader.dataset))) + 1
            print('Train Epoch: {} [{:05d}/{:05d} ({:02.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
            

In [ ]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)


Train Epoch: 1 [00000/60000 (00%)]	Loss: 2.307516
Train Epoch: 1 [00640/60000 (01%)]	Loss: 2.302698
Train Epoch: 1 [01280/60000 (02%)]	Loss: 2.306925
Train Epoch: 1 [01920/60000 (03%)]	Loss: 2.290555
Train Epoch: 1 [02560/60000 (04%)]	Loss: 2.293198
Train Epoch: 1 [03200/60000 (05%)]	Loss: 2.283789
Train Epoch: 1 [03840/60000 (06%)]	Loss: 2.281834
Train Epoch: 1 [04480/60000 (07%)]	Loss: 2.294910
Train Epoch: 1 [05120/60000 (09%)]	Loss: 2.288762
Train Epoch: 1 [05760/60000 (10%)]	Loss: 2.284253
Train Epoch: 1 [06400/60000 (11%)]	Loss: 2.284108
Train Epoch: 1 [07040/60000 (12%)]	Loss: 2.283559
Train Epoch: 1 [07680/60000 (13%)]	Loss: 2.267116
Train Epoch: 1 [08320/60000 (14%)]	Loss: 2.251754
Train Epoch: 1 [08960/60000 (15%)]	Loss: 2.268226
Train Epoch: 1 [09600/60000 (16%)]	Loss: 2.227675
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.209589
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.180263
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.187194
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.111264


Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.095184
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.237732
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.130255
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.107744
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.161814
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.147443
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.335210
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.195709
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.197350
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.206623
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.086560
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.119120
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.247234
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.530244
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.139992
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.066490
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.147412
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.117429
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.167895
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.081822


Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.063579
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.128139
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.050154
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.110234
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.093762
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.083698
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.234583
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.116438
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.082451
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.171975
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.107099
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.082182
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.112403
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.263242
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.093134
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.088393
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.171260
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.120603
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.075770
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.182563


In [ ]:
torch.optim.Optimizer

In [ ]:
type(optimizer)

In [ ]:
isinstance(optimizer, torch.optim.SGD)

In [ ]:
issubclass(type(optimizer), torch.optim.Optimizer)